In [1]:
from src.docker.docker_session_config import DockerSessionConfig
from src.docker.docker_session import docker_session

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
config = DockerSessionConfig(volumes={
    '/Users/Maria.Tigina/PycharmProjects/agents-eval-data/docker': {'bind': '/mnt/vol', 'mode': 'rw'},
    '/Users/Maria.Tigina/PycharmProjects/agents-eval-data/docker_cp': {'bind': '/mnt/vol1', 'mode': 'cp'}
})

In [4]:
with docker_session(config) as session_interface:
    out = session_interface.execute_command("touch /tmp/test.txt")
    print(f"Created file /tmp/test.txt")
    
    out = session_interface.execute_command('ls -la /tmp')
    print(f"List of files in /tmp:\n{out}")
    
    out = session_interface.execute_command("touch /mnt/vol/test_mnt_vol.txt")
    print(f"Created file /mnt/vol/test_mnt_vol.txt")
    
    out = session_interface.execute_command('ls -la /mnt/vol')
    print(f"List of files in /mnt/vol:\n{out}")
    
    session_interface.recreate()
    out = session_interface.execute_command('ls -la /tmp')
    print(f"Recreated container; list of files in /tmp:\n{out}")

Created file /tmp/test.txt
List of files in /tmp:
total 8
drwxrwxrwt 1 root root 4096 Mar  7 10:59 .
drwxr-xr-x 1 root root 4096 Mar  7 10:59 ..
drwxr-xr-x 3 root root   96 Mar  6 11:28 _mnt_vol1
-rw-r--r-- 1 root root    0 Mar  7 10:59 test.txt
Created file /mnt/vol/test_mnt_vol.txt
List of files in /mnt/vol:
total 4
drwxr-xr-x 4 root root  128 Mar  7 10:33 .
drwxr-xr-x 1 root root 4096 Mar  7 10:59 ..
drwxr-xr-x 2 root root   64 Mar  6 13:28 emptyProject
-rw-r--r-- 1 root root    0 Mar  7 10:59 test_mnt_vol.txt
Recreated container; list of files in /tmp:
total 8
drwxrwxrwt 1 root root 4096 Mar  7 10:59 .
drwxr-xr-x 1 root root 4096 Mar  7 10:59 ..
drwxr-xr-x 3 root root   96 Mar  6 11:28 _mnt_vol1


In [5]:
from src.eval.agents.http_agent import HttpAgent

empty_project_path = '/Users/Maria.Tigina/PycharmProjects/agents-eval-data/docker/emptyProject'
docker_config = DockerSessionConfig(
    image='mariatigina/file-system-service:latest',
    command=[],
    working_dir='/app',
    ports={5050: 5050},
    volumes={
        empty_project_path: {"bind": "/project", "mode": "rw"}
    }
)
async def file_system_agent():
    with docker_session(docker_config) as s:
        http_agent = HttpAgent('localhost', '5050')
        print(await http_agent.ping())
        # print(await http_agent.run_command('create_file', {'path': 'hello.txt'}))


In [6]:
await file_system_agent()

ClientOSError: [Errno 54] Connection reset by peer